In [11]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import *

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,KFold,GroupKFold,StratifiedGroupKFold,GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,StackingClassifier,BaggingClassifier
from xgboost import XGBClassifier
from sklearn.utils import shuffle

In [2]:
dataTrain=pd.read_csv("data/Train.txt",sep="#|;",engine="python",header=None,index_col = False)
dataTest=pd.read_csv("data/Test.txt",sep="#|;",engine="python",header=None,index_col = False)
dataCros=pd.read_csv("data/Cross_Validation.txt",sep="#|;",engine="python",header=None,index_col = False)

data=shuffle(pd.concat([dataTrain,dataTest,dataCros]))
data.reset_index(inplace=True)

In [13]:
data.drop("index",axis=1,inplace=True)

In [9]:
data.iloc[data[data[0]==25].index].empty

True

In [14]:
scaler=MinMaxScaler()
X=data.drop(0,axis=1)
X=scaler.fit_transform(X)
Y=data[0]
X_train,X_test,Y_train,Y_test,=train_test_split(X,Y,train_size=0.85)

In [17]:
bool(X_train[])

/tmp/ipykernel_3437/3523834604.py:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  bool(X_train[Y_train[Y_train==25].index])


False

In [32]:
Y_train[Y_train==25].empty

True

In [10]:
Y_xgb=Y_train[Y_train.isin(range(10))]

NameError: name 'Y_train' is not defined

In [89]:
Y_xgb.value_counts()

0    959
7    906
5    868
3    842
6    837
2    832
8    828
9    820
1    810
4    786
Name: 0, dtype: int64

In [8]:
data.dropna(inplace=True,axis=1)

In [26]:
def find_best_model(learning_algorithm,X,Y,expected_accuracy=0.75,model_selection=None,model_selection_params={},params={}):
    import pickle
    while True:
        train_size=np.round(np.random.random(500),2)
        train_size = train_size[(train_size>0.5)&(train_size<0.95)]
        for size in train_size:
            X_train,X_val,y_train,y_val = train_test_split(X,Y,train_size=size)
            
            #adding default parametrs If params is not empty and model_selection is not None
            if model_selection:
                for (key,value) in params.items():
                    model = learning_algorithm()
                    l=set()
                    if hasattr(value,"__iter__") and type(value)!=str:
                        l=set(value)
                    else:
                        l.add(value)
                    l.add(model.__dict__.get(key))
                    params[key]=list(l)
                selected_model = model_selection(model,params,scoring="accuracy",**model_selection_params)
                model = selected_model.fit(X_train,y_train)
            else:
                model = learning_algorithm(**params)
                model.fit(X_train,y_train)
            
            y_val_predict=model.predict(X_val)
            test_accuracy=accuracy_score(y_val_predict,y_val)

            print(f"Train Size: {size}, Model: {learning_algorithm.__name__}, Test Accuracy:{test_accuracy:.3}")
            if test_accuracy>=expected_accuracy:
                with open(f"{learning_algorithm.__name__.lower()}1.pkl","wb") as file:
                    pickle.dump(model,file)
                return test_accuracy,model


In [27]:
accuracy,model=find_best_model(DecisionTreeClassifier,X_train,Y_train,expected_accuracy=0.78)

Train Size: 0.93, Model: DecisionTreeClassifier, Test Accuracy:0.768
Train Size: 0.53, Model: DecisionTreeClassifier, Test Accuracy:0.738
Train Size: 0.86, Model: DecisionTreeClassifier, Test Accuracy:0.768
Train Size: 0.59, Model: DecisionTreeClassifier, Test Accuracy:0.715
Train Size: 0.65, Model: DecisionTreeClassifier, Test Accuracy:0.729
Train Size: 0.76, Model: DecisionTreeClassifier, Test Accuracy:0.754
Train Size: 0.66, Model: DecisionTreeClassifier, Test Accuracy:0.751
Train Size: 0.67, Model: DecisionTreeClassifier, Test Accuracy:0.745
Train Size: 0.75, Model: DecisionTreeClassifier, Test Accuracy:0.766
Train Size: 0.64, Model: DecisionTreeClassifier, Test Accuracy:0.748
Train Size: 0.84, Model: DecisionTreeClassifier, Test Accuracy:0.754
Train Size: 0.68, Model: DecisionTreeClassifier, Test Accuracy:0.746
Train Size: 0.56, Model: DecisionTreeClassifier, Test Accuracy:0.738
Train Size: 0.73, Model: DecisionTreeClassifier, Test Accuracy:0.754
Train Size: 0.69, Model: DecisionT

In [46]:
params={"solver":['sag', 'saga'],"max_iter":[400,500],"n_jobs":4}
accuracy,model=find_best_model(LogisticRegression,X_train,Y_train,expected_accuracy=0.944,params={"solver":"saga"})

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train Size: 0.88, Model: LogisticRegression, Test Accuracy:0.935


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train Size: 0.87, Model: LogisticRegression, Test Accuracy:0.931


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train Size: 0.8, Model: LogisticRegression, Test Accuracy:0.939


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train Size: 0.7, Model: LogisticRegression, Test Accuracy:0.936


KeyboardInterrupt: 

In [49]:
accuracy,model=find_best_model(RandomForestClassifier,X_train,Y_train,expected_accuracy=0.968)

Train Size: 0.89, Model: RandomForestClassifier, Test Accuracy:0.965
Train Size: 0.74, Model: RandomForestClassifier, Test Accuracy:0.971


In [53]:
import pickle
with open("randomforestclassifier1.pkl","rb") as file:
    model=pickle.load(file)
y_predict=model.predict(X_test)
accuracy_score(y_predict,Y_test)
y_predict[y_predict==23]
X_test[np.where(y_predict==24)]
bool(X_test[np.where(y_predict==24)])

/tmp/ipykernel_3437/1683013678.py:8: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  bool(X_test[np.where(y_predict==24)])


False

In [ ]:
D=data[data[0].isin(range(10))]
scaler=MinMaxScaler()
X=D.drop(0,axis=1)
X=scaler.fit_transform(X)
Y=D[0]
X_xgb_train,X_xgb_test,Y_xgb_train,Y_xgb_test=train_test_split(X,Y,train_size=0.85)
accuracy,model=find_best_model(XGBClassifier,X_xgb_train,Y_xgb_train,expected_accuracy=0.975)

Train Size: 0.52, Model: XGBClassifier, Test Accuracy:0.964


In [10]:
rm=RandomForestClassifier()
rm.fit(X_train,Y_train)

RandomForestClassifier()

In [11]:
y_predict=rm.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score,recall_score,precision_score

In [13]:
accuracy_score(y_predict,Y_test)

0.9692765113974232

In [14]:
recall_score(y_predict,Y_test,average="macro")

0.9689573950656234

In [15]:
precision_score(y_predict,Y_test,average="macro")

0.9701006474162567

In [16]:
y_Cpredict=rm.predict(X_cross)
accuracy_score(y_Cpredict,Y_cross)

0.9693069306930693

In [17]:
import numpy as np
np.arange(5,100)

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [18]:
np.random.sample(50)

array([0.24120147, 0.26093858, 0.25761032, 0.31313136, 0.97715322,
       0.49214637, 0.23259958, 0.53224467, 0.97085207, 0.42100153,
       0.57118193, 0.36219839, 0.76414985, 0.69586097, 0.81481816,
       0.65290262, 0.18283472, 0.42955509, 0.72562087, 0.63832127,
       0.3610112 , 0.93799135, 0.8973366 , 0.51704504, 0.11112429,
       0.36604807, 0.45529201, 0.59993654, 0.2838712 , 0.28074111,
       0.89682178, 0.75817502, 0.96430512, 0.08333195, 0.28192978,
       0.30086238, 0.94112373, 0.4297987 , 0.63403275, 0.32427923,
       0.54002206, 0.59165639, 0.80885427, 0.46048997, 0.74869232,
       0.79684164, 0.18698933, 0.4032263 , 0.86668174, 0.72952692])

In [19]:
from xgboost import XGBClassifier

In [22]:
xg=XGBClassifier()
xg.fit(X_train,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [26]:
Y_predict=xg.predict(X_test)

In [27]:
accuracy_score(Y_predict,Y_test)

0.9534192269573836

In [28]:
from sklearn.metrics import classification_report

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
LogisticRegression().__dict__

{'penalty': 'l2',
 'dual': False,
 'tol': 0.0001,
 'C': 1.0,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'class_weight': None,
 'random_state': None,
 'solver': 'lbfgs',
 'max_iter': 100,
 'multi_class': 'auto',
 'verbose': 0,
 'warm_start': False,
 'n_jobs': None,
 'l1_ratio': None}

In [33]:
with open("randomforest.pkl","rb") as file:
    model=pickle.load(file)

In [35]:
Y_predict=model.predict(X_test)

In [36]:
accuracy_score(Y_test,Y_predict)

0.9920713577799801

In [44]:
with open("logistic_regression.pkl","w") as file:
    pickle.dump(model,file)

FileNotFoundError: [Errno 2] No such file or directory: 'logistic_regression.pkl'

In [4]:
import pickle

In [7]:
with open("xgbclassifier.pkl","rb") as file:
    model=pickle.load(file)

In [13]:
D=data[data[0].isin(range(11))]
X_xgb=D.drop(0,axis=1)
Y_xgb=D[0]

In [14]:
y_predict= model.predict(X_xgb)

In [17]:
print(classification_report(Y_xgb,y_predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       985
           1       0.98      0.98      0.98       842
           2       0.99      0.99      0.99       864
           3       0.99      0.99      0.99       880
           4       0.98      0.99      0.99       814
           5       0.99      0.99      0.99       907
           6       0.99      1.00      0.99       871
           7       0.99      0.99      0.99       944
           8       1.00      0.99      1.00       869
           9       0.98      0.99      0.99       862
          10       1.00      0.99      0.99       814

    accuracy                           0.99      9652
   macro avg       0.99      0.99      0.99      9652
weighted avg       0.99      0.99      0.99      9652



In [12]:
y_predict

array([ 5,  0, 10, ...,  4,  4,  4])

In [18]:
pip install pytube


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 148.0 kB/s eta 0:00:00 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Number of videos in playlist: 0


Found existing installation: pytube 12.1.3
Uninstalling pytube-12.1.3:
  Would remove:
    /home/yasin/.local/bin/pytube
    /home/yasin/.local/lib/python3.10/site-packages/pytube-12.1.3.dist-info/*
    /home/yasin/.local/lib/python3.10/site-packages/pytube/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [81]:
import pickle
with open("randomforest.pkl","rb") as file:
    model = pickle.load(file)

In [83]:
len(X_test)

19382

In [60]:
np.concatenate((np.arange(10),np.arange(20,30)))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29])